In [1]:
import numpy as np

def projection_matrix(theta1, theta2):
    """
    Construct the 4x4 projection matrix for polarizers at angles theta1 and theta2.
    """
    P_theta1 = np.array([[np.cos(theta1)**2, np.cos(theta1) * np.sin(theta1)],
                         [np.cos(theta1) * np.sin(theta1), np.sin(theta1)**2]])
    
    P_theta2 = np.array([[np.cos(theta2)**2, np.cos(theta2) * np.sin(theta2)],
                         [np.cos(theta2) * np.sin(theta2), np.sin(theta2)**2]])
    
    return np.kron(P_theta1, P_theta2)

def coincidence_probability(state_or_rho, theta1, theta2):
    """
    Calculate the coincidence probability for a given state (vector or density matrix)
    and polarizer angles theta1 and theta2.
    """
    P = projection_matrix(theta1, theta2)
    
    if state_or_rho.ndim == 1:  # State vector
        probability = np.dot(state_or_rho.conj().T, np.dot(P, state_or_rho)).real
    elif state_or_rho.ndim == 2:  # Density matrix
        probability = np.trace(np.dot(state_or_rho, P)).real
    else:
        raise ValueError("Input must be either a state vector or a density matrix.")
    
    return probability

def expectation_value(state_or_rho, theta1, theta2):
    """
    Calculate the expectation value E(theta1, theta2) for a given state or density matrix.
    """
    P_plusplus = coincidence_probability(state_or_rho, theta1, theta2)  # N_++
    P_plusminus = coincidence_probability(state_or_rho, theta1, theta2 + np.pi / 2)  # N_+-
    P_minusplus = coincidence_probability(state_or_rho, theta1 + np.pi / 2, theta2)  # N_-+
    P_minusminus = coincidence_probability(state_or_rho, theta1 + np.pi / 2, theta2 + np.pi / 2)  # N_--
    
    # E(theta1, theta2) from coincidence counts
    E = (P_plusplus + P_minusminus - P_plusminus - P_minusplus) / (P_plusplus + P_minusminus + P_plusminus + P_minusplus)
    return E

def calculate_chsh_s(state_or_rho):
    """
    Calculate the S value for the CHSH inequality using a given state or density matrix.
    """
    # Measurement angles for CHSH inequality
    angles = [
        (0, np.pi/8),            # θ1, θ2
        (0, 3*np.pi/8),          # θ1, θ2'
        (np.pi/4, np.pi/8),      # θ1', θ2
        (np.pi/4, 3*np.pi/8)     # θ1', θ2'
    ]
    
    # Calculate the expectation values for each angle pair
    E_values = [expectation_value(state_or_rho, theta1, theta2) for (theta1, theta2) in angles]
    
    # Calculate the S value
    S = np.abs(E_values[0] - E_values[1] + E_values[2] + E_values[3])
    return S

# Example Usage

# Define the maximally entangled Bell state |phi+> = (|HH> + |VV>)/sqrt(2)
phi_plus = 1/np.sqrt(2) * np.array([1, 0, 0, 1])

# Define the density matrix for |phi+>
rho_phi_plus = np.outer(phi_plus, phi_plus.conj())

# Calculate the S value for the pure state vector
S_pure = calculate_chsh_s(phi_plus)
print(f"S value (pure state): {S_pure}")

# Calculate the S value for the density matrix
S_mixed = calculate_chsh_s(rho_phi_plus)
print(f"S value (density matrix): {S_mixed}")


S value (pure state): 2.8284271247461903
S value (density matrix): 2.82842712474619


In [8]:
alphas = []
Ss = []
for alpha in np.linspace(0, 1, 20):
    alphas.append(alpha)
    rho = np.zeros((4,4))
    rho[0,0] = 1
    rho[3,3] = 1
    rho[0, 3] = alpha
    rho[3, 0] = alpha
    rho *= 0.5
    S = calculate_chsh_s(rho)
    Ss.append(S)
    
fig, ax = plt.subplots()
ax.plot(alphas, Ss)
fig.show()

In [2]:
Ps = []
Ss = []
for P in np.linspace(0, 1, 20):
    Ps.append(P)
    rho = np.zeros((4,4))
    rho[0,0] = P
    rho[3,3] = 1 - P 
    S = calculate_chsh_s(rho)
    Ss.append(S)
    

In [5]:

fig, ax = plt.subplots()
ax.plot(Ps, Ss, '.')
fig.show()